## Import stuff

In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import random
from PIL import Image
import logging
import time
import sklearn.metrics as metrics


## Load training data from csv


In [3]:
val_data = pd.read_csv("val_distances.csv",index_col=0)
val_labels = pd.read_csv("val_labels.txt", header=None)


val_data.drop("anchor_left_distance_mn", axis=1, inplace=True)
val_data.drop("anchor_right_distance_mn", axis=1, inplace=True)
for extractor in ["xl","resnet","clip","clip2"]:
    val_data[extractor] = val_data["anchor_left_distance_"+extractor]-val_data["anchor_right_distance_"+extractor]
    val_data.drop("anchor_left_distance_"+extractor, axis=1, inplace=True)
    val_data.drop("anchor_right_distance_"+extractor, axis=1, inplace=True)


X_val = np.array(val_data.iloc[:, :].values,  dtype=float)
Y_val = np.array(val_labels.iloc[:, :].values,  dtype=float).reshape(-1)

print("X shape: ", X_val.shape)
print("Y shape: ", Y_val.shape)

val_data

X shape:  (7476, 4)
Y shape:  (7476,)


,xl,resnet,clip,clip2
0,-0.376129,-0.326017,-0.570965,-2.990695
1,-0.122618,-0.079811,-0.140376,0.274413
2,0.261849,0.122114,-0.125301,-2.006118
3,-0.547212,-0.657696,-1.082755,-5.145643
4,0.644307,0.251229,0.540372,1.023604
...,...,...,...,...
7471,0.105163,-0.299347,-0.152062,-0.437958
7472,0.105163,-0.299347,-0.152062,-0.437958
7473,0.105163,-0.299347,-0.152062,-0.437958
7474,0.105163,-0.299347,-0.152062,-0.437958


In [17]:
pred = []
for row in X_val:
    score_left = row[0]+row[1]+row[2]
    pred.append(0 if score_left>0 else 1)

pred = np.array(pred)
accuracy = metrics.accuracy_score(Y_val, pred<0.5)
accuracy

0.7804975922953451

In [16]:
pred = []
for row in X_val:
    score_left = 0
    score_left += row[0]>0
    score_left += row[1]>0
    score_left += row[2]>0
    pred.append(0 if score_left>1 else 1)

pred = np.array(pred)
accuracy = metrics.accuracy_score(Y_val, pred<0.5)
accuracy

0.7766185125735687